In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd

# Import your CRUD Python module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Setup the MongoDB connection
username = "aacuser"
password = "SNHU1234"

# Instantiate AnimalShelter with connection parameters
db = AnimalShelter(
    username=username,
    password=password,
    host="nv-desktop-services.apporto.com",
    port=33059,
    db="AAC",
    collection="animals"
)

# Load all data from MongoDB into a DataFrame
try:
    df = pd.DataFrame.from_records(db.read({}))
    df.drop(columns=['_id'], inplace=True)  # Drop the '_id' column to prevent crashes
except Exception as e:
    print(f"Error loading data from MongoDB: {e}")
    df = pd.DataFrame()

#########################
# Dashboard Layout / View
#########################
# Encode and embed Grazioso Salvare’s logo
image_filename = '/home/tylerlowe_snhu/Downloads/Grazioso Salvare Logo.png' 
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
except FileNotFoundError:
    print(f"Error: Logo file not found at {image_filename}")
    encoded_image = None

app = JupyterDash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.Div([
        # Grazioso Salvare Logo
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()) if encoded_image else '',
            style={'height': '100px', 'width': 'auto', 'display': 'block', 'margin': '0 auto'}
        ),
        html.Center(html.B(html.H1('CS-340 Dashboard - Tyler Lowe'))),
    ]),
    html.Hr(),
    html.Div([
        # Interactive Filtering Options
        html.Label("Filter by Rescue Type:"),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'  # Default value
        )
    ], style={'width': '50%', 'margin': '0 auto'}),
    html.Hr(),
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ] if not df.empty else [],
        data=df.to_dict('records') if not df.empty else [],
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0],
        style_data={'whiteSpace': 'normal'},
        style_cell={'textAlign': 'left'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        # Pie Chart
        html.Div(id='graph-id', className='col s12 m6'),
        # Geolocation Map
        html.Div(id='map-id', className='col s12 m6')
    ]),
    html.Br(),
    html.Hr(),
    # Unique Identifier
    html.Div(
        "Project Two - Tyler Lowe",
        style={'textAlign': 'center', 'fontSize': '18px', 'fontWeight': 'bold'}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# Update data table based on filter selection
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    try:
        # MongoDB queries for filtering
        if filter_type == 'Water Rescue':
            query = {"breed": {"$in": ["Labrador Retriever", "Newfoundland"]}}
        elif filter_type == 'Mountain or Wilderness Rescue':
            query = {"breed": {"$in": ["German Shepherd", "Bernese Mountain Dog"]}}
        elif filter_type == 'Disaster or Individual Tracking':
            query = {"breed": {"$in": ["Bloodhound", "Beagle"]}}
        else:
            query = {}  # Reset

        # Fetch data and convert to DataFrame
        filtered_data = db.read(query)
        df = pd.DataFrame.from_records(filtered_data)

        # Handle missing columns and unsupported types
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)  # Remove ObjectId
        df.fillna("", inplace=True)  # Replace NaN values with empty strings
        for col in df.columns:
            df[col] = df[col].astype(str)  # Ensure all values are strings
        
        return df.to_dict('records')
    except Exception as e:
        print(f"Error updating data table: {e}")
        return []

# Update pie chart based on table data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    try:
        if viewData is None or len(viewData) == 0:
            return "No data to display."

        dff = pd.DataFrame.from_dict(viewData)
        return [
            dcc.Graph(
                figure=px.pie(
                    dff,
                    names='breed',
                    title='Preferred Breeds Distribution'
                )
            )
        ]
    except Exception as e:
        print(f"Error updating pie chart: {e}")
        return "Error displaying chart."

# Update geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    try:
        if viewData is None or len(viewData) == 0:
            return "No data available to display."

        dff = pd.DataFrame.from_dict(viewData)
        row = index[0] if index else 0  # Default to first row if no selection

        # Ensure latitude and longitude are converted to float
        latitude = float(dff.iloc[row, dff.columns.get_loc('location_lat')])
        longitude = float(dff.iloc[row, dff.columns.get_loc('location_long')])
        breed = dff.iloc[row, dff.columns.get_loc('breed')]
        name = dff.iloc[row, dff.columns.get_loc('name')]

        if pd.isnull(latitude) or pd.isnull(longitude):
            latitude, longitude = 30.75, -97.48  # Default to Austin, TX

        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[latitude, longitude], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(
                           position=[latitude, longitude],
                           children=[
                               dl.Tooltip(breed if breed else "Unknown Breed"),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(name if name else "No Name")
                               ])
                           ]
                       )
                   ])
        ]
    except Exception as e:
        print(f"Error updating map: {e}")
        return "Error displaying map."

# Run the app
app.run_server(debug=True)


Connection to MongoDB successful!
Dash app running on http://127.0.0.1:29242/
Error updating map: could not convert string to float: ''
